In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falconlin/Development/trading
Changed current working directory to: /Users/falconlin/Development/trading
Initialized project with base directory: /Users/falconlin/Development/trading


In [2]:
from constants.coin_names import CoinNames
from settings import DATA_DIR
import pandas as pd
import os
dfs = {}
no_data = []
for coin in CoinNames.__members__:
    p = DATA_DIR / f'{coin}.csv'
    if os.path.exists(p):
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        dfs[coin] = df
    else:
        no_data.append(coin)
else:
    os.sys.stderr.write(f'No data for {no_data}')

No data for ['APE', 'RLB', 'HPOS', 'UNIBOT', 'COMP', 'FXS', 'SNX', 'OX', 'FRIEND', 'ZRO', 'BLZ', 'BANANA', 'FTT', 'TRB', 'CANTO', 'NTRN', 'KAS', 'BSV', 'TON', 'POLYX', 'GAS', 'AXL', 'PENDLE', 'STG', 'STRAX', 'BADGER', 'ZEN', 'ILV', 'MAV', 'NFTI', 'SUPER', 'USTC', 'RSR', 'CAKE', 'REQ', 'MAVIA', 'W', 'PANDORA', 'TAO']

In [3]:
dfs["SOL"].iloc[-1, :]["tic"]

1709942400000

In [4]:
from collections import Counter


def get_majority_shape(dfs):
    # Get the shapes of all DataFrames
    shapes = [df.shape for df in dfs.values()]
    
    # Count occurrences of each shape
    shape_counts = Counter(shapes)
    
    # Display votes for each shape
    for shape, count in shape_counts.items():
        print(f"Shape: {shape}, Votes: {count}")
    
    # Find the most common shape
    majority_shape, majority_votes = shape_counts.most_common(1)[0]
    
    return majority_shape

def get_majority_start_end(dfs):
    # Get the shapes of all DataFrames
    start_end = [(df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"] ) for df in dfs.values()]
    
    # Count occurrences of each shape
    start_end_counts = Counter(start_end)
    
    # Display votes for each shape
    for se, count in start_end_counts.items():
        print(f"Start End: {se}, Votes: {count}")
    
    # Find the most common shape
    majority_se, majority_votes = start_end_counts.most_common(1)[0]
    
    return majority_se

major_shape = get_majority_shape(dfs)
major_se = get_majority_start_end(dfs)

Shape: (2881, 11), Votes: 56
Shape: (1832, 11), Votes: 1
Shape: (1761, 11), Votes: 1
Shape: (891, 11), Votes: 1
Shape: (296, 11), Votes: 1
Start End: (1707350400000, 1709942400000), Votes: 56
Start End: (1708294500000, 1709942400000), Votes: 1
Start End: (1708358400000, 1709942400000), Votes: 1
Start End: (1709141400000, 1709942400000), Votes: 1
Start End: (1709676900000, 1709942400000), Votes: 1


In [5]:
def filter_dfs(dfs, shape, start_end):
    valid_dfs = {}
    for coin, df in dfs.items():
        if df.shape != shape:
            print(f"Shape mismatch for {coin}. Expected {shape}, got {df.shape}")
            continue
        elif (df.iloc[0, :]["tic"], df.iloc[-1, :]["tic"]) != start_end:
            print(f"Start/End mismatch for {coin}. Expected {start_end}, got {df.iloc[0, :]['tic'], df.iloc[-1, :]['tic']}")
            continue
        else:
            valid_dfs[coin] = df

    return valid_dfs


valid_dfs = filter_dfs(dfs, major_shape, major_se)

Shape mismatch for AI. Expected (2881, 11), got (1832, 11)
Shape mismatch for PIXEL. Expected (2881, 11), got (1761, 11)
Shape mismatch for AR. Expected (2881, 11), got (891, 11)
Shape mismatch for MYRO. Expected (2881, 11), got (296, 11)


In [8]:
df = valid_dfs["SOL"]

df.head()

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc
symbol,,,,,,,,,,,
SOL,2024-02-08-00:00:00,2024-02-08-00:14:59,15m,17,101.41,101.42,100.97,101.19,63.59,1707350400000,1707351299999
SOL,2024-02-08-00:15:00,2024-02-08-00:29:59,15m,22,102.03,102.03,101.40,101.52,33.99,1707351300000,1707352199999
SOL,2024-02-08-00:30:00,2024-02-08-00:44:59,15m,17,101.78,102.19,101.78,102.19,683.71,1707352200000,1707353099999
SOL,2024-02-08-00:45:00,2024-02-08-00:59:59,15m,13,101.99,101.99,101.53,101.79,30.61,1707353100000,1707353999999
SOL,2024-02-08-01:00:00,2024-02-08-01:14:59,15m,22,101.29,102.00,101.25,102.00,139.06,1707354000000,1707354899999


In [11]:
# Calculate the percentage change in the 'close' column for each coin
df["pct_change"] = df.groupby(level=0)["close"].pct_change() * 100
df

,start_time,end_time,interval,number_of_trades,close,high,low,open,volume,tic,toc,pct_change
symbol,,,,,,,,,,,,
SOL,2024-02-08-00:00:00,2024-02-08-00:14:59,15m,17,101.41,101.42,100.97,101.19,63.59,1707350400000,1707351299999,NaN
SOL,2024-02-08-00:15:00,2024-02-08-00:29:59,15m,22,102.03,102.03,101.40,101.52,33.99,1707351300000,1707352199999,0.611380
SOL,2024-02-08-00:30:00,2024-02-08-00:44:59,15m,17,101.78,102.19,101.78,102.19,683.71,1707352200000,1707353099999,-0.245026
SOL,2024-02-08-00:45:00,2024-02-08-00:59:59,15m,13,101.99,101.99,101.53,101.79,30.61,1707353100000,1707353999999,0.206327
SOL,2024-02-08-01:00:00,2024-02-08-01:14:59,15m,22,101.29,102.00,101.25,102.00,139.06,1707354000000,1707354899999,-0.686342
...,...,...,...,...,...,...,...,...,...,...,...,...
SOL,2024-03-08-23:00:00,2024-03-08-23:14:59,15m,15,144.93,144.93,144.65,144.67,53.07,1709938800000,1709939699999,0.520183
SOL,2024-03-08-23:15:00,2024-03-08-23:29:59,15m,17,145.28,145.98,145.24,145.26,1555.54,1709939700000,1709940599999,0.241496
SOL,2024-03-08-23:30:00,2024-03-08-23:44:59,15m,21,144.64,145.39,144.51,145.27,602.50,1709940600000,1709941499999,-0.440529


In [24]:
_df = pd.concat(valid_dfs.values(), keys=valid_dfs.keys(), names=["coin"])
_df.info()

_df.to_csv(DATA_DIR / "all.csv", index=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 161336 entries, ('SOL', 'SOL') to ('DYM', 'DYM')
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   start_time        161336 non-null  object 
 1   end_time          161336 non-null  object 
 2   interval          161336 non-null  object 
 3   number_of_trades  161336 non-null  int64  
 4   close             161336 non-null  float64
 5   high              161336 non-null  float64
 6   low               161336 non-null  float64
 7   open              161336 non-null  float64
 8   volume            161336 non-null  float64
 9   tic               161336 non-null  int64  
 10  toc               161336 non-null  int64  
 11  pct_change        2880 non-null    float64
dtypes: float64(6), int64(3), object(3)
memory usage: 15.1+ MB


In [21]:

_df.head()

start_time             end_time interval  \
coin symbol                                                      
SOL  SOL     2024-02-08-00:00:00  2024-02-08-00:14:59      15m   
     SOL     2024-02-08-00:15:00  2024-02-08-00:29:59      15m   
     SOL     2024-02-08-00:30:00  2024-02-08-00:44:59      15m   
     SOL     2024-02-08-00:45:00  2024-02-08-00:59:59      15m   
     SOL     2024-02-08-01:00:00  2024-02-08-01:14:59      15m   

             number_of_trades   close    high     low    open  volume  \
coin symbol                                                             
SOL  SOL                   17  101.41  101.42  100.97  101.19   63.59   
     SOL                   22  102.03  102.03  101.40  101.52   33.99   
     SOL                   17  101.78  102.19  101.78  102.19  683.71   
     SOL                   13  101.99  101.99  101.53  101.79   30.61   
     SOL                   22  101.29  102.00  101.25  102.00  139.06   

                       tic            toc  pct_change  
coin symbol                                            
SOL  SOL     1707350400000  1707351299999         NaN  
     SOL     1707351300000  1707352199999    0.611380  
     SOL     1707352200000  1707353099999   -0.245026  
     SOL     1707353100000  1707353999999    0.206327  
     SOL     1707354000000  1707354899999   -0.686342